In [1]:
import numpy as np
import numpy.random as nr
from scipy.stats import truncnorm
import statsmodels.api as sm
import pandas as pd

## matplotlib with display of graphs inline
import matplotlib.pyplot as plt
import seaborn as sns
#matplotlib inline

In [2]:
## Create data set as multivariate Normal
covariance = np.array([[1.0,0.7,0.7,0.0],
                      [0.7,1.0,0.6,0.7],
                      [0.7,0.6,1.0,0.2],
                      [0.0,0.7,0.2,1.0]])
effect_data = pd.DataFrame(nr.multivariate_normal(mean=[3.0,2.0,2.0,2.0], cov=covariance, size=500),
                           columns=['Fan','TimePlaying','SocialMedia','GameFamilarity'])

## Truncate values to range 0.0 <= x <= 10.0
effect_data[effect_data < 0.0] = 0.0
effect_data[effect_data > 10.0] = 10.0

In [3]:
 ## Square the values of TimePlaying to give positive skew
effect_data['TimePlaying'] = np.square(effect_data['TimePlaying'])

## And round all values to 2 decimal places
effect_data = effect_data.round(decimals=2)
effect_data

,Fan,TimePlaying,SocialMedia,GameFamilarity
0,1.44,0.04,0.00,1.14
1,4.43,10.18,3.61,2.26
2,3.92,5.55,2.30,1.50
3,4.51,8.89,2.16,1.60
4,5.42,22.66,4.19,3.57
5,3.27,2.90,2.56,1.42
6,1.81,1.11,1.36,1.60
7,4.27,8.05,1.73,1.83
8,4.80,13.79,1.62,2.43
9,2.45,0.74,1.83,0.75


In [4]:
social_time_model = sm.OLS(effect_data['SocialMedia'],effect_data['TimePlaying']).fit()
social_time_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:            SocialMedia   R-squared (uncentered):                   0.738
Model:                            OLS   Adj. R-squared (uncentered):              0.737
Method:                 Least Squares   F-statistic:                              1403.
Date:                Sun, 05 Jan 2020   Prob (F-statistic):                   4.49e-147
Time:                        12:38:20   Log-Likelihood:                         -776.82
No. Observations:                 500   AIC:                                      1556.
Df Residuals:                     499   BIC:                                      1560.
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
TimePlaying     0.2840      0.008     37.452      0.000       0.269       0.299
==============================================================================
Omnibus:                       26.210   Durbin-Watson:                   1.619
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               30.959
Skew:                          -0.502   Prob(JB):                     1.89e-07
Kurtosis:                       3.691   Cond. No.                         1.00
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [5]:
social_time_fan_model = sm.OLS(effect_data['SocialMedia'],effect_data[['TimePlaying','Fan']]).fit()
social_time_fan_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:            SocialMedia   R-squared (uncentered):                   0.906
Model:                            OLS   Adj. R-squared (uncentered):              0.906
Method:                 Least Squares   F-statistic:                              2400.
Date:                Sun, 05 Jan 2020   Prob (F-statistic):                   1.98e-256
Time:                        12:38:21   Log-Likelihood:                         -520.14
No. Observations:                 500   AIC:                                      1044.
Df Residuals:                     498   BIC:                                      1053.
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
TimePlaying     0.0412      0.009      4.418      0.000       0.023       0.059
Fan             0.5900      0.020     29.872      0.000       0.551       0.629
==============================================================================
Omnibus:                        3.660   Durbin-Watson:                   1.870
Prob(Omnibus):                  0.160   Jarque-Bera (JB):                3.344
Skew:                           0.135   Prob(JB):                        0.188
Kurtosis:                       2.704   Cond. No.                         5.12
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [6]:
social_time_familarity_model = sm.OLS(effect_data['SocialMedia'],effect_data[['TimePlaying','GameFamilarity']]).fit()
social_time_familarity_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:            SocialMedia   R-squared (uncentered):                   0.773
Model:                            OLS   Adj. R-squared (uncentered):              0.772
Method:                 Least Squares   F-statistic:                              846.5
Date:                Sun, 05 Jan 2020   Prob (F-statistic):                   6.19e-161
Time:                        12:38:21   Log-Likelihood:                         -740.91
No. Observations:                 500   AIC:                                      1486.
Df Residuals:                     498   BIC:                                      1494.
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
TimePlaying        0.1679      0.015     11.190      0.000       0.138       0.197
GameFamilarity     0.3854      0.044      8.770      0.000       0.299       0.472
==============================================================================
Omnibus:                        4.177   Durbin-Watson:                   1.866
Prob(Omnibus):                  0.124   Jarque-Bera (JB):                4.029
Skew:                          -0.217   Prob(JB):                        0.133
Kurtosis:                       3.070   Cond. No.                         6.80
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [7]:
social_fan_model = sm.OLS(effect_data['SocialMedia'],effect_data[['Fan']]).fit()
social_fan_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:            SocialMedia   R-squared (uncentered):                   0.902
Model:                            OLS   Adj. R-squared (uncentered):              0.902
Method:                 Least Squares   F-statistic:                              4610.
Date:                Sun, 05 Jan 2020   Prob (F-statistic):                   3.35e-254
Time:                        12:38:21   Log-Likelihood:                         -529.76
No. Observations:                 500   AIC:                                      1062.
Df Residuals:                     499   BIC:                                      1066.
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Fan            0.6662      0.010     67.896      0.000       0.647       0.685
==============================================================================
Omnibus:                        5.442   Durbin-Watson:                   1.866
Prob(Omnibus):                  0.066   Jarque-Bera (JB):                4.298
Skew:                           0.121   Prob(JB):                        0.117
Kurtosis:                       2.616   Cond. No.                         1.00
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""